<img align=right src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150>
<img src=images/acme.jpeg  width=100 align="left">

# Vault Enterprise
<img src=images/vault-enterprise1.png width="500">

Find the full list of Enterprise features at https://www.hashicorp.com/products/vault/pricing/

Prereq: 
You need to have Vault Enterprise binary and appropriate License file

## Setup

In [ ]:
/bin/rm -rf $HOME/demos/vault_cluster/vrd/data/*

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
which evault
evault --version


### Populate Vault server configuration file (hcl or json format)
Configures:
* storage
* listener
* ui
* telemetry
* seal
* etc

In [ ]:
mkdir -p $HOME/demo/vault_cluster/vrd/data/raft
cat <<EOF > $HOME/demos/vault_cluster/vrd/vrd_config.hcl
storage "raft" {
  path = "$HOME/demos/vault_cluster/vrd/data"
  node_id = "demo"
}
listener "tcp" {
  address = "0.0.0.0:8200"
  cluster_address = "0.0.0.0:8201"
  tls_disable = "true"
}
api_addr = "https://127.0.0.1:8200"
cluster_addr = "https://127.0.0.1:8201"
disable_mlock="false"
disable_cache="false"
ui = "true"
max_lease_ttl="24h"
default_lease_ttl="1h"
raw_storage_endpoint=true
cluster_name="hashi-vault"
insecure_tls="true"
plugin_directory="$HOME/demos/vault_cluster/vrd/data/plugins"
EOF

\
**EXECUTE the following command in SEPARATE window**

```
vaulte server -config=$HOME/demos/vault_cluster/vrd/vrd_config.hcl
```

In [ ]:
vault status

##### Notice Initialized and Sealed status
When a Vault server is started, it starts in a **sealed** state and it does not know how to decrypt data.  Before any operation can be performed on the Vault, it must be unsealed.  

## Initialize Vault

Initialization is the process of configuring the Vault:
* Encryption key gets generated 
* Unseal keys are created
* Initial root token is setup

Note: the initialization takes few seconds to complete.

<img src=images/shamir_secret_sharing.png width=500>



In [ ]:
curl --request POST --data '{"secret_shares": 1, "secret_threshold": 1}' $VAULT_ADDR/v1/sys/init > ~/demos/vault_cluster/vrd/vault_init_output

**The above command is the same as**  
```
vault operator init -key-shares=1 -key-threshold=1 |tee  ~/demos/vault_cluster/vrd/vault_init_output
```


##### Parse output to variables (for demo use)

In [ ]:
export VAULT_ROOT_TOKEN=$(cat ~/demos/vault_cluster/vrd/vault_init_output | jq -r '.root_token')
export VAULT_KEY_BASE64=$(cat ~/demos/vault_cluster/vrd/vault_init_output | jq -r '.keys_base64[0]' )
echo $VAULT_ROOT_TOKEN
echo $VAULT_KEY_BASE64

---
## Unseal Vault
Unsealing  is the process of constructing the master key necessary to decrypt the data encryption keys and read the data.  It requires a threshold of unseal keys (see the Intialize Vault above). Unsealing is a manual process and can become tedious when you have multiple Vault clusters! Hint: explore/use Auto-Unseal feature.

Seal the Vault is an ultimate **break glass** procedure.  For example, you suspect that someone has unauthorized access to Vault with wrong intention.

In [ ]:
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data '{ "key": "'$VAULT_KEY_BASE64'" }' \
    $VAULT_ADDR/v1/sys/unseal

## Apply License to Vault

In [ ]:
sleep 5
curl \
    --insecure \
    --header "X-Vault-Token: $VAULT_ROOT_TOKEN" \
    --request PUT \
    --data @/Users/tio/Documents/vault_license.json \
    $VAULT_ADDR/v1/sys/license

### Vault Auto Unseal
Vault supports opt-in automatic unsealing via cloud technologies: AliCloud KMS, Amazon KMS, Azure Key Vault and Google Cloud KMS.  This feature delegates the unsealing process to trusted cloud providers to ease operation.

## Interacting with Vault

Vault **HTTP API**
* Full access to Vault via HTTP
* Every aspect of Vault can be controlled via API


Vault **CLI**
* uses HTTP API to access Vault
* it is a thin wrapper around the HTTP API
* outputs are formatted

**Note**: You should remember **-output-curl-string** CLI flag to find out the `curl` equivalent of CLI.

In [ ]:
vault secrets list -output-curl-string

### Getting help

In [ ]:
vault server -h

### View Vault via a browser
Open a web browser and visit http://127.0.0.1:8200/ui/vault
Use **TOKEN** and **Sign in**

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">